In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
pd.options.display.max_columns = None
pd.options.display.max_rows = 500
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
import warnings
import regex as re
warnings.filterwarnings('ignore')
from decimal import *
TWOPLACES = Decimal(10) ** -2
from natsort import natsorted
import ast

from enrich_dimensions.rounds import rounds, rounding, re_extract, curate, round_string_float, clean_data,reg_ex     
from enrich_dimensions.params import parameters, query_from_file
from enrich_dimensions.query_file import query_from_file 
from enrich_dimensions.custom import custom_field 

In [2]:
# import json
# pand = pd.read_csv('Data/wbm_length.csv')

In [3]:

# # df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
# # custom_field_df = pd.json_normalize(df['custom_fields'])
# # fields = ['Width [Nom]','Overall Width [Nom]']
# # df = pd.concat([df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)


# # df_custom=df[(df['Capacity (Weight) [Nom]'].astype(str)!='[]')&(df['Capacity (Weight) [Nom]'].astype(str)!='nan')&(df['Weight Capacity'].astype(str)!='[]')&(df['Weight Capacity'].astype(str)!='nan')]
# df_custom=pand[(pand['Length [Nom]'].astype(str)!='[]')&(pand['Length [Nom]'].astype(str)!='nan')]
# df_cur=pand[(pand['Length [Nom]'].astype(str)=='[]')|(pand['Length [Nom]'].astype(str)=='nan')]
# # df_custom

In [4]:
# df_custom['Length [Nom]']

In [5]:
# parameters
customer_id = '90'
formatted_attribute = 'Length'
customer_name='%wbmason%'
buckets = """Office Chairs"""

strategy_version_input=762
attribute_id_input=2425

In [6]:
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')
value='%n/a%'
params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [7]:
# def regex_patterns(unit):
#     return fr'(?i)(\d+\.\d+[\W]?[\W]?[\W]{unit}\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]{unit}\b)|((?<!\.)(?<!\/)\d+(?!\.\d)[\W]?[\W]?[\W]{unit}\b)|()'
# print(regex_patterns('Le?n?g?t?h?'))
# print(regex_patterns('He?i?g?h?t?'))
# print(regex_patterns('Wi?d?t?h?'))
# print(regex_patterns('Diam?e?t?e?r?'))

In [8]:
# buckets = [x for x in enrich_db(f'''SELECT * FROM (SELECT * FROM strategy_buckets sb WHERE strategy_version_id = {strategy_version_input}) AS active_strat
# INNER JOIN strategy_buckets_attributes sba ON active_strat.id = sba.strategy_bucket_id
# WHERE sba.attribute_id = {attribute_id_input}
# AND sba.status = 'OPT_IN' ''')['bucket_id'].unique()]
# # buckets

In [9]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
    
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))    

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1

def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))

def replace_all(text, dic):
    for i, j in dic.items():
        text = text.replace(i, j)
    return text

def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]

def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()

def round_string_float(x):
    try:
        return remove_exponent(Decimal(x).quantize(TWOPLACES))
    except:
        return x  
    
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    
#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))

from fractions import Fraction
def frac2string(s):
    i, f = s.groups(0)
    f = round_string_float(Fraction(f))
    return str(int(i) + round_string_float(float(f)))
# measure='This is a fraction: 100 1/64'
# [re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, measure)]

import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A

In [10]:
import json
df = query_from_file(file_name='./query/custom_fields.sql', params=params)
# df['custom_fields'] = df['custom_fields'].apply(lambda x: json.loads(x))
custom_field_df=pd.json_normalize(df['custom_fields'])
# custom_field_df = pd.json_normalize(df['custom_fields'])
fields = [ 'Length']
length = pd.concat([df.drop('custom_fields', axis=1), custom_field_df[fields]], axis = 1)
print(len(df))

808


In [11]:
df_cust=length[(length['Length'].astype(str)!='[]')&(length['Length'].astype(str)!='nan')&(length['Length'].astype(str)!='NaN')&(length['Length'].astype(str)!='Assorted')]
# length[(length['Length'].astype(str)!='[]')&(length['Length'].astype(str)!='nan')&(length['Length'].astype(str)!='NaN')]
df_cust['match']=df_cust['Length'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)]).apply(lambda x: re.sub(r'\"',' in',str(x))).apply(lambda x: re.sub(r"\'",'"',str(x)))
df_cust['match'].explode().value_counts()
df_cust[curation_col]=df_cust['match']
matchcust=df_cust[['external_id',curation_col]]
print(len(matchcust))
# df_cust

13


In [12]:
del length['product_id']
del length['bucket_id']
del length['image_url']
del length['value']
# # del length['custom_fields']

In [13]:
del df['custom_fields']

In [14]:
df['buckets'].explode().value_counts()[0:10]

Rugs & Mats                                        284
Envelopes/Mailers                                   89
Food Storage Canisters/Containers & Accessories     69
Folders & File Bags                                 40
Greeting Cards, Postcards, & Invitations            36
Boxes, Baskets, & Bins                              33
Presentation Boards                                 25
Ribbons, Bows, & Embellishments                     17
Cleaning Tools & Accessories                        14
Binders & Accessories                               13
Name: buckets, dtype: int64

In [15]:
df_cur=length[(length['Length'].astype(str)=='[]')|(length['Length'].astype(str)=='nan')|(length['Length'].astype(str)=='NaN')]
print(len(df_cur))

795


In [16]:
label=df_cur[df_cur['buckets'].astype(str)=='Labels, Stickers & Tickets']

dia=r'''(?i)(\d+\"?.{0,3}x\s?\d+)|()'''
label['match']=label['long_desc'].apply(lambda x: re_extract(dia, str(x)))

print(len(label[label['match'].astype(str)=='[]']))
stick=label[label['match'].astype(str)=='[]']

dia=r'''(?i)(\d+(?:\.\d+|.\d+\/\d+)?\"?\W{0,3}dia)|((?<!\/)(?<!\d)(?:\.\d+|.\d+\/\d+)?\"?\W{0,3}dia)|()'''
stick['match']=stick['product_name'].apply(lambda x: re_extract(dia, str(x)))
print(len(stick[stick['match'].astype(str)!='[]']))
sticks=stick[stick['match'].astype(str)!='[]']


sticks['matches']=sticks['match'].apply(lambda x: re.sub(r'(?i)\"?"?\s?Dia|�\s?Dia',' in',str(x))).apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
sticks['matches']=sticks['matches'].apply(lambda x: re.sub(r'(?i)\"?\s?Diam|�\s?Diam',' in',str(x))).apply(lambda x: re.sub(r'•','',str(x)))
# stick['matches']=stick['matches'].apply(lambda x: re.sub(r'Assorted|Letter|Legal|Up to ','',str(x))).apply(lambda x: re.sub(r'"|(?<!\s)in',' in',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'0.5 in\, 0.75 in','["0.5 in","0.75 in"]',str(x)))
# stick['matches']=stick['matches'].apply(lambda x: re.sub(r'1000','1,000',str(x))).apply(lambda x: re.sub(r'3500','3,500',str(x)))
sticks['matches']=sticks['matches'].apply(lambda x: re.sub(r'''\[\'|\'\]''','',str(x))).apply(lambda x: re.sub(r'''('\s?,\s?')''','","',str(x))).apply(lambda x: re.sub(r"\'",'',str(x)))
# sticks
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
sticks['matchez']=sticks['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))

sticks['matchez']=sticks['matchez'].apply(lambda x: re.sub(r'''\[''','["',str(x))).apply(lambda x: re.sub(r'''\]''','"]',str(x))).apply(lambda x: re.sub(r'''('\s?,\s?')''','","',str(x))).apply(lambda x: re.sub(r"\'",'',str(x)))
sticks[curation_col]=sticks['matchez']
matchdiam=sticks[['external_id',curation_col]]
matchdiam[curation_col].explode().value_counts()
# label

1
0


Series([], Name: Q:length, dtype: int64)

In [17]:
dia=r'''(?i)(\d+(?:\.\d+|.\d+\/\d+)?\"?\W{0,3}dia)|((?<!\/)(?<!\d)(?:\.\d+|.\d+\/\d+)?\"?\W{0,3}dia)|()'''
stick['match']=stick['product_name'].apply(lambda x: re_extract(dia, str(x)))
# print(len(stick[stick['match'].astype(str)=='[]']))
circle=stick[stick['match'].astype(str)=='[]']

dia=r'''(?i)(Circle Tags, \d+")|(\d+(?:\.\d+|.?\d+\/\d+)?" Circle)|()'''
circle['match']=circle['product_name'].apply(lambda x: re_extract(dia, str(x)))
print(len(circle[circle['match'].astype(str)!='[]']))
circles=circle[circle['match'].astype(str)!='[]']


circles['matches']=circles['match'].apply(lambda x: re.sub(r'(?i)\"\s?circle',' in',str(x))).apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
circles['matches']=circles['matches'].apply(lambda x: re.sub(r'(?i)Circle Tags, ','',str(x))).apply(lambda x: re.sub(r'''(?i)\[\'\[\\\'|\[\"\[\'''','["',str(x))).apply(lambda x: re.sub(r'''(?i)\\\'\]\'\]|\'\]\"\]''','"]',str(x)))
circles['matches']=circles['matches'].apply(lambda x: re.sub(r'(?<!\[)\"(?!\])',' in',str(x)))
# stick['matches']=stick['matches'].apply(lambda x: re.sub(r'Assorted|Letter|Legal|Up to ','',str(x))).apply(lambda x: re.sub(r'"|(?<!\s)in',' in',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'0.5 in\, 0.75 in','["0.5 in","0.75 in"]',str(x)))
circles['matches']=circles['matches'].apply(lambda x: re.sub(r'\.625','.63',str(x)))
# circles['matches']=circles['matches'].apply(lambda x: re.sub(r'''\[\'|\'\]''','',str(x))).apply(lambda x: re.sub(r'''('\s?,\s?')''','","',str(x))).apply(lambda x: re.sub(r'''\[\\|\\''','',str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# circles['matchez']=circles['matches'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))

# circles['matchez']=circles['matchez'].apply(lambda x: re.sub(r'''\[''','["',str(x))).apply(lambda x: re.sub(r'''\]''','"]',str(x))).apply(lambda x: re.sub(r'''('\s?,\s?')''','","',str(x)))


circles[curation_col]=circles['matches']
matchcircle=circles[['external_id',curation_col]]
# matchcircle
circles

0


Empty DataFrame
Columns: [buckets, external_id, product_name, long_desc, url, Length, match, matches, Q:length]
Index: []

In [18]:
##### papers=length[length['buckets'].astype(str)=='Papers']

dia=r'''(?i)(x\s?\d+(?:.?\d+\/\d+|\.\d+)?\s?(?:ft|inch|"|'|foot|yd)?)|()'''
papers=length[length['buckets'].astype(str)=='Papers']
papers['match']=papers['product_name'].apply(lambda x: re_extract(dia, str(x)))
print(len(papers[papers['match'].astype(str)!='[]']))
# papers[papers['match'].astype(str)=='[]']
pap=papers[papers['match'].astype(str)!='[]']
pap['match']=pap['match'].apply(lambda x: re.sub(r'"','in',str(x))).apply(lambda x: re.sub(r'x ','',str(x))).apply(lambda x: re.sub(r"\[\'|\[in",'["',str(x))).apply(lambda x: re.sub(r"\'\]|in\]",'"]',str(x))).apply(lambda x: re.sub(r"\'\s?,\s?\'",'","',str(x))).apply(lambda x: re.sub(r"\'",' ft',str(x)))
pap['match']=pap['match'].apply(lambda x: re.sub(r'in',' in',str(x))).apply(lambda x: re.sub(r'(?<!a-z)\"\]',' in"]',str(x))).apply(lambda x: re.sub(r'ft in','ft',str(x))).apply(lambda x: re.sub(r'in in','in',str(x))).apply(lambda x: re.sub(r'(?<![a-z])","',' in","',str(x))).apply(lambda x: re.sub(r'\[\[','[',str(x))).apply(lambda x: re.sub(r'\]\]',']',str(x)))#.apply(lambda x: re.sub(r'1000','1,000',str(x)))                      
pap['match']=pap['match'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# pap['match']=pap['match'].apply(lambda x: re.sub(r'in',' in',str(x))).apply(lambda x: re.sub(r'(?<!a-z)"\]','in"]',str(x)))

# pap['match']=pap['match'].apply(lambda x: re.sub(r'''\[\"\[\'''','["',str(x))).apply(lambda x: re.sub(r'''\'\]\"\]''','"]',str(x))).apply(lambda x: re.sub(r"','",'","',str(x)))#.apply(lambda x: re.sub(r"'",' ft',str(x))).apply(lambda x: re.sub(r'x ','',str(x)))

pap[curation_col]=pap['match'].apply(lambda x: re.sub(r"\[\'\[",'[',str(x))).apply(lambda x: re.sub(r"\]\'\]",']',str(x)))
matchpaperfour=pap[['external_id',curation_col]]
matchpaperfour
# pap['match'].explode().value_counts()

12


external_id     Q:length
35  5637406456    ["11 in"]
36  5637406224    ["11 in"]
37  5637406999    ["14 in"]
38  5637497735    ["14 in"]
40  5637588026    ["85 ft"]
41  5637488531    ["11 in"]
42  5637488532    ["11 in"]
43  5637588072  ["2500 ft"]
44  5637588055    ["80 ft"]
45  5637588060    ["85 ft"]
46  5637588062   ["127 ft"]
47  5637588015    ["90 ft"]

In [19]:
labels=length[length['buckets'].astype(str)=='Labels, Stickers & Tickets']

dia=r'''(?i)(5.?1\/2"?\s?i?n?c?h?\shigh)|(5.?5"?\s?i?n?c?h?\shigh)|(\d+(?:\.\d+)?(.\d+\/\d+)\s?i?n?c?h?\shigh)|(\d.{0,3}diam)|(\d.{0,3}x\s?\d+.?)|(\d.{0,3}expan)|()'''
length['eigth']=length['product_name'].apply(lambda x: re_extract(dia, str(x)))

dia=r'''(?i)(\d+\s?(?:\d+\/\d+|\.\d+).{0,2}x.?\d+\s?(?:\d+\/\d+|\.\d+)?.{0,2})|()'''
length['paper']=length['product_name'].apply(lambda x: re_extract(dia, str(x)))

length['eigth'].explode().value_counts()
print(len(length[length['buckets'].astype(str)=='Binders & Accessories'][length['paper'].astype(str)!='[]']))
binder=length[length['buckets'].astype(str)!='Binders & Accessories'][length['paper'].astype(str)!='[]']#['paper'].explode().value_counts()[0:500]
binder['paper']=binder['paper'].apply(lambda x: re.sub(r'''�''',' ',str(x))).apply(lambda x: re.sub(r'''\,.*|"|\[\'|w|\'\]|h''','',str(x)))
binder['paper']=binder['paper'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# binder['paper'].explode().value_counts()[0:500]

dia=r'''(?i)(8.5\s?x)|()'''
binder['split']=binder['paper'].apply(lambda x: re_extract(dia, str(x)))
eight=binder[binder['split'].astype(str)!='[]']
eight[curation_col]='8.5 in'

6


In [20]:
dia=r'''(?i)(8.5\s?x)|()'''
binder['split']=binder['paper'].apply(lambda x: re_extract(dia, str(x)))
eight=binder[binder['split'].astype(str)!='[]']
eight[curation_col]='8.5 in'


In [21]:
inches=r'''(?i)(8\.5.{0,3}x\s?11)|(8.?1\/2.{0,3}x\s?11)|()'''
length['eigth']=length['long_desc'].apply(lambda x: re_extract(inches, str(x)))
print(len(length[length['eigth'].astype(str)!='[]']))
eight=length[length['eigth'].astype(str)!='[]']
eight[curation_col]='11 in'
matcheight=eight[['external_id',curation_col]]
# matcheight

26


In [22]:
labels=length[length['buckets'].astype(str)=='Labels, Stickers & Tickets']

dia=r'''(?i)(?i)(8\.5.{0,3}x\s?11)|(8.?1\/2.{0,3}x\s?11\s?.{0,4})|()'''
length['eigth']=length['long_desc'].apply(lambda x: re_extract(dia, str(x)))
length['eigth'].explode().value_counts()
notebook=length[length['buckets'].astype(str)=='Notebooks, Journals, & Writing Pads'][length['eigth'].astype(str)!='[]']
notebook[curation_col]='11.75 in'
matchnote=notebook[['external_id',curation_col]]
# matchnote

In [23]:
labels=length[length['buckets'].astype(str)=='Paper']

dia=r'''(?i)(5.?1\/2"?\s?i?n?c?h?\shigh)|(5.?5"?\s?i?n?c?h?\shigh)|(\d+(?:\.\d+)?(.\d+\/\d+)\s?i?n?c?h?\shigh)|(\d.{0,3}diam)|(\d.{0,3}x\s?\d)|(\d.{0,3}expan)|()'''
length['eigth']=length['product_name'].apply(lambda x: re_extract(dia, str(x)))

dia=r'''(?i)(11.{0,5}x.?)|(x.?11.{0,5})|()'''
length['paper']=length['product_name'].apply(lambda x: re_extract(dia, str(x)))

length['eigth'].explode().value_counts()
print(len(length[length['buckets'].astype(str)=='Binders & Accessories'][length['paper'].astype(str)!='[]']))
paper=length[length['buckets'].astype(str)=='Binders & Accessories'][length['paper'].astype(str)!='[]']
paper[curation_col]='17 in'
matchpaper=paper[['external_id',curation_col]]
paper

5


buckets external_id  \
5  Binders & Accessories  5637485853   
6  Binders & Accessories  5637485854   
7  Binders & Accessories  5637485855   
8  Binders & Accessories  5637485856   
9  Binders & Accessories  5637516566   

                                                            product_name  \
5                  JAM Paper Display Book, 8 1/2" x 11", Black, 12 Pages   
6      JAM Paper Spiral Ring Display Book, 8 1/2" x 11", Black, 12 Pages   
7  JAM Paper Smart View Presentation Book, 8 1/2" x 11", Black, 12 Pages   
8      JAM Paper Spiral Ring Display Book, 8 1/2" x 11", Black, 36 Pages   
9  JAM Paper Display Book, 8 1/2" x 11", Black, 48 Pages per Book, 12/PK   

                                                                                                                                                                                                                                long_desc  \
5                  Display Book, 8 1/2" x 11", Black, 12 Pages•Sturdy and durable build•Contains plastic and clear sheets•Easy to remove and insert material•Has a smooth finish•Ideal for displaying projects, photos, reports, and more   
6      Spiral Ring Display Book, 8 1/2" x 11", Black, 12 Pages•Sturdy and durable build•Contains plastic and clear sheets•Easy to remove and insert material•Has a smooth finish•Ideal for displaying projects, photos, reports, and more   
7  Smart View Presentation Book, 8 1/2" x 11", Black, 12 Pages•Sturdy and durable build•Contains plastic and clear sheets•Easy to remove and insert material•Has a smooth finish•Ideal for displaying projects, photos, reports, and more   
8      Spiral Ring Display Book, 8 1/2" x 11", Black, 36 Pages•Sturdy and durable build•Contains plastic and clear sheets•Easy to remove and insert material•Has a smooth finish•Ideal for displaying projects, photos, reports, and more   
9  Display Book, 8 1/2" x 11", Black, 48 Pages per Book, 12/PK•Sturdy and durable build•Contains plastic and clear sheets•Easy to remove and insert material•Has a smooth finish•Ideal for displaying projects, photos, reports, and more   

                                                            url Length  \
5        https://wbmason.com/ProductDetail.aspx?ItemID=JAMF12BK    NaN   
6    https://wbmason.com/ProductDetail.aspx?ItemID=JAM521312118    NaN   
7  https://wbmason.com/ProductDetail.aspx?ItemID=JAM78910215142    NaN   
8    https://wbmason.com/ProductDetail.aspx?ItemID=JAM521312131    NaN   
9       https://wbmason.com/ProductDetail.aspx?ItemID=JAMF48BKB    NaN   

      eigth        paper Q:length  
5  [2" x 1]  [x 11", Bl]    17 in  
6  [2" x 1]  [x 11", Bl]    17 in  
7  [2" x 1]  [x 11", Bl]    17 in  
8  [2" x 1]  [x 11", Bl]    17 in  
9  [2" x 1]  [x 11", Bl]    17 in

In [24]:
labels=length[length['buckets'].astype(str)=='Labels, Stickers & Tickets']

dia=r'''(?i)(5.?1\/2"?\s?i?n?c?h?\shigh)|(5.?5"?\s?i?n?c?h?\shigh)|(\d+(?:\.\d+)?(.\d+\/\d+)\s?i?n?c?h?\shigh)|(\d.{0,3}diam)|(\d.{0,3}x\s?\d)|(\d.{0,3}expan)|()'''
length['eigth']=length['product_name'].apply(lambda x: re_extract(dia, str(x)))
length['eigth'].explode().value_counts()
length[length['buckets'].astype(str)=='Folders & File Bags']#[length['eigth'].astype(str)=='[]']

buckets external_id  \
721  Folders & File Bags  5637483449   
722  Folders & File Bags  5637483085   
723  Folders & File Bags  5637483109   
724  Folders & File Bags  5637483111   
725  Folders & File Bags  5637483134   
726  Folders & File Bags  5637483061   
727  Folders & File Bags  5637483077   
728  Folders & File Bags  5637491539   
729  Folders & File Bags  5637499181   
730  Folders & File Bags  5637497759   
731  Folders & File Bags  5637503938   
732  Folders & File Bags  5637505673   
733  Folders & File Bags  5637505711   
734  Folders & File Bags  5637505712   
735  Folders & File Bags  5637510345   
736  Folders & File Bags  5637525480   
737  Folders & File Bags  5637511562   
738  Folders & File Bags  5637511565   
739  Folders & File Bags  5637603367   
740  Folders & File Bags  5637510281   
741  Folders & File Bags  5637510282   
742  Folders & File Bags  5637506197   
743  Folders & File Bags  5637506204   
744  Folders & File Bags  5637524163   
745  Folders & File Bags  5637524051   
746  Folders & File Bags  5637524063   
747  Folders & File Bags  5637524064   
748  Folders & File Bags  5637524067   
749  Folders & File Bags  5637524068   
750  Folders & File Bags  5637524074   
751  Folders & File Bags  5637524075   
752  Folders & File Bags  5637524076   
753  Folders & File Bags  5637483472   
754  Folders & File Bags  5637603365   
755  Folders & File Bags  5637603366   
756  Folders & File Bags  5637506205   
757  Folders & File Bags  5637506210   
758  Folders & File Bags  5637506230   
759  Folders & File Bags  5637506237   
760  Folders & File Bags  5637506238   

                                                                                                                          product_name  \
721  JAM Paper Plastic Expansion Envelopes with Hook & Loop Closure, Letter Open End, 9 3/4" x 11 3/4", 1" Expansion, Dark Blue, 12/PK   
722           JAM Paper Plastic Envelopes with Button & String Tie Closure, Legal Booklet, 9 3/4" x 14 1/2", Black Translucent, 108/PK   
723                              JAM Paper Plastic Envelopes with Snap Closure, Letter Booklet, 9 3/4" x 13", Light Purple Grid, 12/PK   
724                                      JAM Paper Plastic Envelopes with Snap Closure, Letter Booklet, 9 3/4" x 13", Pink Grid, 12/PK   
725       JAM Paper Plastic Expansion Envelopes with Hook & Loop Closure, Letter Booklet, 8 5/8" x 11 1/2", 1" Expansion, Lilac, 12/PK   
726                     JAM Paper Plastic Envelopes with Button & String Tie Closure, Index Size, 5 1/2" x 7 1/2", Lilac Purple, 12/PK   
727                 JAM Paper Plastic Envelopes with Button & String Tie Closure, Legal Booklet, 9 3/4" x 14 1/2", Lilac Purple, 12/PK   
728                  JAM Paper Plastic Accordion Folder, 6 Pocket Expanding File with Elastic Closure, Coupon (5 1/4" x 7"), Baby Pink   
729                                           JAM Paper Plastic Business Card Holder Case, Clear Black National Building Museum Design   
730                                                JAM Paper Plastic Envelope with Snap Closure and Leather Edge, 10" x 12 1/2", Green   
731                                                                             JAM Paper Sliding Snap Business Card Holder Case, Teal   
732                            JAM Paper Plastic Envelopes with Button & String Tie Closure, Square, 13" x 13", Metallic Purple, 12/PK   
733                                 JAM Paper Plastic Envelopes with Tuck Flap Closure, Letter Open-End, 9 1/2" x 11 1/2", Pink, 12/PK   
734                            JAM Paper Plastic Envelopes with Hook & Loop Closure, Letter Booklet, 9 3/4" x 13", Fuchsia Pink, 12/PK   
735                                                               JAM Paper Plastic Business Card Holder Case, 2 1/4" x 3 3/4", Purple   
736                                                                                     JAM Paper Swing Lock Report Cover Folder, Blue   
737                           

In [25]:
# print(len(length[length['eigth'].astype(str)=='[]']))

# inches=r'''(?i)(.{0,10}\d+.{0,3}x\s?\d+.{0,10})|(\d+["'](?!.Cap))|()'''
# length['general']=length['long_desc'].apply(lambda x: re_extract(inches, str(x)))

# print(len(length[(length['buckets'].astype(str)=='Binders & Accessories')&(length['general'].astype(str)=='[]')][0:500]))
# match=length[(length['buckets'].astype(str)=='Binders & Accessories')&(length['general'].astype(str)=='[]')
# match[curation_col]='n/a'
# matchone=match[['external_id',curation_col]]
# # matchone

In [26]:
# custom_field_df.columns.to_list()

In [27]:
# print(len(custom_field_df[(custom_field_df['Sheet Size (W x H)'].astype(str)!='[]')&(custom_field_df['Sheet Size (W x H)'].astype(str)!='nan')]))
# custom_field_df[(custom_field_df['Sheet Size (W x H)'].astype(str)!='[]')&(custom_field_df['Sheet Size (W x H)'].astype(str)!='nan')]

In [28]:
# print(len(length[(length['buckets'].astype(str)=='Labels, Stickers & Tickets')&(length['general'].astype(str)!='[]')]))
# length[(length['buckets'].astype(str)=='Labels, Stickers & Tickets')&(length['general'].astype(str)!='[]')][0:500]

In [29]:
# df = query_from_file(file_name='./query/uncurated.sql', params=params)
# df = df[df['bucket_id'].isin(buckets)]
# len(df)

In [30]:

# df_custom['match_cust']=df_custom['Length [Nom]'].apply(lambda x: re.sub(r'\s\s',' ',str(x))).apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'\.00','',str(x))).apply(lambda x: re.sub(r'\.10','.1',str(x))).apply(lambda x: re.sub(r'\.20','.2',str(x))).apply(lambda x: re.sub(r'\.30','.3',str(x))).apply(lambda x: re.sub(r'\.40','.4',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\.60','.6',str(x))).apply(lambda x: re.sub(r'\.70','.7',str(x))).apply(lambda x: re.sub(r'\.80','.8',str(x))).apply(lambda x: re.sub(r'\.90','.9',str(x)))
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'Assorted|Letter|Legal|Up to ','',str(x))).apply(lambda x: re.sub(r'"|(?<!\s)in',' in',str(x))).apply(lambda x: re.sub(r'in\.','in',str(x))).apply(lambda x: re.sub(r'ft\.','ft',str(x))).apply(lambda x: re.sub(r'0.5 in\, 0.75 in','["0.5 in","0.75 in"]',str(x)))
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'1000','1,000',str(x))).apply(lambda x: re.sub(r'3500','3,500',str(x)))
# df_custom['match_cust']=df_custom['match_cust'].apply(lambda x: re.sub(r'''\[\'|\'\]''','',str(x)))
# df_custom[curation_col]=df_custom['match_cust']
# regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|(\d+\.\d0)|(\,)|([a-eghj-mo-su-z])|(\.(?!\d))|(\d\d\d\d)|()'          
# df_custom['rounding'] = df_custom[curation_col].apply(lambda x: re_extract(regex_pattern_rounding, x))
# df_custom[df_custom['rounding'].astype(str)!='[]']

In [31]:
# df_custom[curation_col]=df_custom[curation_col].apply(lambda x: str(f'["{x}"]'))
# matchcustom=df_custom[['external_id',curation_col]]
# matchcustom[0:500]

In [32]:
# del df['product_id']
# del df['attributes']
# del df['customer_id']
# del df['bucket_id']
# del df['values']

### Find Inches and feet

In [33]:
inches=r'''(?i)(\d+\W?(?:(?:inc?h?))|(?:")|(?:”)|(?:(?<!\w)'{2})|(?:''(?!\w)))|()'''
feet=r'''(?i)(\b\d+\W?(?:(?:fe?e?t))|(?:foot)|(?:(?<!')(?!\w)'(?!\w)(?!')))|()'''
df['inches']=df['long_desc'].apply(lambda x: re_extract(inches, str(x)))
df['feet']=df['long_desc'].apply(lambda x: re_extract(feet, str(x)))

In [34]:
# regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
# df['matches_height'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
# regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
# df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))

# regex_pattern_blank=r'\d'
# df['new_matches'] = df['matches_length'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [35]:
regex_pattern_length=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)\d+(?:(?:\.\d+)?(?:\D?\d+\/\d+)?)[\W]{0,3}Le?n?g?t?h?\b)|()'''         
df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))
regex_pattern_height=r'(?i)(\d+\.\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|(\d+.?\d+\/\d+[\W]?[\W]?[\W]?He?i?g?h?t?\b)|((?<!\.)(?<!up.to.)(?<![\w])(?<!\/)(?<!\/)\d+(?!\.\d)(?<!\d\\?\"?\-\d\d)[\W]?[\W]?[\W]?He?i?g?h?t?(?!.?setting)(?!.?position)(?!.?adjust)(?!.?-)(?!.?100%)\b)|()'        
df['matches_height'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_height, x))
regex_pattern_width=r'(?i)(\d+\.\d+.?.?.?Wi?d?t?h?\b)|(\d+.?\d+\/\d+.?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Wi?d?t?h?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Wi?d?t?h?\b)|()'
df['matches_width'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_width, x))
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))
regex_pattern_ft=r"\d'.?.?.?[LWHD]"
df['matches_ft'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_ft, x))
regex_pattern_blank=r'\d'
df['new_matches'] = df['matches_length'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
regex_pattern_blank=r'''(?i)(\d+\s?x\s?\d+)|(\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:"?i?n?c?h?'?'?f?o?o?t?f?e?e?t?”?).?x.?\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:"?i?n?c?h?'?'?f?o?o?t?f?e?e?t?”?)?.?x?.?\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:"?i?n?c?h?'?'?f?o?o?t?f?e?e?t?”?))|\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:"?i?n?c?h?'?'?f?o?o?t?f?e?e?t?”?)|()'''            
df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

# Length

In [36]:
regex_pattern_length=r'''(?i)((?<!\d)(?<!\w)(?<!\.)(?<!\/)(?<!-)\d+(?:(?:\.\d+)?(?:\D?\d+\/\d+)?)[\W]{0,3}Le?n?g?t?h?\b)|()'''         
df['matches_length'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_length, x))

In [37]:
heights=df[(df['matches_length'].astype(str)!='[]')]
# unit=r'''(?i)(\binc?h?)|(")|(”)|((?<!\w)'{2}(?!\w))|((?<!\w)'')|(feet)|(foot)|((?<!')(?!\w)'(?!\w)(?<!'))|(ft)|()'''
# heights['unit']=heights['matches_length'].apply(lambda x: re_extract(unit, str(x)))
heights=heights[(heights['inches'].astype(str)!='[]')|(heights['feet'].astype(str)!='[]')]
# del heights['unit']
heights['height_final']=heights['matches_length'].apply(lambda x: re.sub(r'''(?i)((?:\\)?(?:(?:\")|(?:\”)|(?:inc?h?)|(?:''))(?:[^']?)(Length|L))|(\\ in)|((?<!\w)'')|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'''(?i)(\\?(?:(?:(?<!')'(?!'))|(?:ft)|(?:foot)|(?:feet))\W?(Length|L))''',' ft',str(x))).apply(lambda x: re.sub(r'''(?i)(\\ in)|("L)''',' in',str(x)))#.apply(lambda x: re.sub(r'''(\'.?xa.{0,3}in ft)''',' ft',str(x)))#.apply(lambda x: re.sub(r'(?i)\"xa0W|xa0W',' in',str(x)))#.apply(lambda x: re.sub(r'''(?i)\'\\xa in''',' ft',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r"'",'"',str(x))).apply(lambda x: re.sub(r'" ','"',str(x))).apply(lambda x: re.sub(r'""','"',str(x))).apply(lambda x: re.sub(r'''(?i)(\\")''','"',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'''(?i)(\["\[")''','["',str(x))).apply(lambda x: re.sub(r'''(?i)("]"])''','"]',str(x))).apply(lambda x: re.sub(r', "',',"',str(x)))
heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'''(?i)(\["\[\\")''','["',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
heights['height_final']=heights['height_final'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
heights['height_final']=heights['height_final'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('(?i)height|\\?"\W?H',' in',str(x))).apply(lambda x: re.sub('(?i)in.?in',' in',str(x)))

# del heights['matches_length']
# del heights['matches_width']
# del heights['matches_diameter']
# del heights['matches_ft']
# del heights['new_matches']
# del heights['blank_matches']

heights['height_final']=heights['height_final'].apply(lambda x: re.sub(r'\.625','63',str(x))).apply(lambda x: re.sub(r'\.188','19',str(x))).apply(lambda x: re.sub(r'12\: in','12 in',str(x))).apply(lambda x: re.sub(r'''\\\\"\\\\xa\s?in''',' in',str(x))).apply(lambda x: re.sub(r'''  ''',' ',str(x))).apply(lambda x: re.sub(r'''inn''','in',str(x))).apply(lambda x: re.sub(r'''\?|•''','',str(x))).apply(lambda x: re.sub(r'''ln''',' in',str(x)))
regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|()'  
print(len(heights))
heights[curation_col]=heights['height_final']

heights[curation_col]=heights[curation_col].apply(lambda x: re.sub(r'2000','2,000',str(x))).apply(lambda x: re.sub(r'1200','1,200',str(x))).apply(lambda x: re.sub(r'\.50','.5',str(x))).apply(lambda x: re.sub(r'\[\"\d+L\"\]','',str(x)))

regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|(\d+\.\d0)|(\,)|([a-eghj-mo-su-z])|(\.(?!\d))|(\d\d\d\d)|()'          
heights['rounding'] = heights[curation_col].apply(lambda x: re_extract(regex_pattern_rounding, x))
heights[heights['rounding'].astype(str)!='[]']

57


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, inches, feet, matches_length, matches_height, matches_width, matches_diameter, matches_ft, new_matches, blank_matches, blank_matches_long, height_final, Q:length, rounding]
Index: []

In [58]:
print(len(heights))

matchlength=heights[['external_id',curation_col]]


57


# Height

In [39]:
regex_pattern_diameter=r'(?i)(\d+\.\d+.?.?.?Diam?e?t?e?r?\b)|(\d+.?\d+\/\d+.?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.\d).?.?.?Diam?e?t?e?r?\b)|((?<!\.)(?<!\/)\d+(?!\.).?.?.?Diam?e?t?e?r?\b)|()'
df['matches_diameter'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_diameter, x))

In [40]:
height_new=df[(df['matches_length'].astype(str)!='[]')&(df['feet'].astype(str)=='[]')&(df['matches_diameter'].astype(str)=='[]')]#&(df['new_matches'].astype(str)!='[]')]
height_new=height_new[(height_new['inches'].astype(str)!='[]')&(height_new['feet'].astype(str)!='[]')]

height_new['matches_length']=height_new['matches_length'].apply(lambda x: natsorted(x))
height_new['matches_length']=height_new['matches_length'].apply(lambda x: re.sub(r'''(?i)((?:\\)?(?:(?:\")|(?:\”)|(?:inc?h?)|(?:''))(?:[^']?)(Length|L))|(\\ in)|(square\W?Inch)''',' in',str(x))).apply(lambda x: re.sub(r'\\" L',' in',str(x))).apply(lambda x: re.sub(r'\\" L|"L|\"xa0L|xa0L|"|�|•','',str(x))).apply(lambda x: re.sub(r'''(?i)(\\?(?:(?:(?<!')'(?!'))|(?:ft)|(?:foot)|(?:feet))\W?(Length|L))''',' ft',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))                               
height_new['matches_length']=height_new['matches_length'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
height_new['matches_length']=height_new['matches_length'].apply(lambda x: re.sub('\\\" H',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
import ast
def remove_duplicates(A):
    [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
    return A
height_new['matches_length']=height_new['matches_length'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
height_new['matches_length']=height_new['matches_length'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
# del height_new['matches_width']
del height_new['matches_diameter']

height_new['matches_length']=height_new['matches_length'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
height_new['matches_length']=height_new['matches_length'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
height_new['matches_length']=height_new['matches_length'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

regex_pattern_rounding=r'(?i)(\d+\.\d\d\d)|()'  
print(len(height_new))
height_new['rounding'] = height_new['matches_length'].apply(lambda x: re_extract(regex_pattern_rounding, x))
height_new[height_new['rounding'].astype(str)!='[]']

0


Empty DataFrame
Columns: [buckets, bucket_id, product_id, external_id, product_name, long_desc, url, image_url, value, inches, feet, matches_length, matches_height, matches_width, matches_ft, new_matches, blank_matches, blank_matches_long, rounding]
Index: []

In [41]:
height_new[curation_col]=height_new['matches_length']
matchesheight=height_new[['external_id',curation_col]]
print(len(matchesheight))
# matchesheight

0


# Length-Good Stuff

In [42]:
# from natsort import natsorted
# regex_pattern=r'(?i)((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})\d+\.\d+[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})\d+.?\d+\/\d+[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!\.)(?<!\/)\d+(?!\.\d)[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|((?<!Spacing between.{0,16})(?<!Cord Measures.{0,10})(?<!\.)(?<!\/)\d+(?!\.)[\W]{0,2}"[\W]{0,2}(?!let)(?!leg)Le?n?g?t?h?\b)|()'           
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern , x))
# print(len(df[df['matches'].astype(str)!='[]']))
# df['matches']=df['matches'].apply(lambda x: natsorted(x))

In [43]:
# regex_pattern_blank=r'\d'
# df['new_matches'] = df['matches'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# df_new=df[(df['new_matches'].astype(str)!='[]')&(df['matches'].astype(str)!='[]')]
# df_new['matches']=df_new['matches'].apply(lambda x: re.sub(r'(?i)\\" Le?n?g?t?h?|"Le?n?g?t?h?|Le?n?g?t?h?',' in',str(x))).apply(lambda x: re.sub(r'(?i)\\"\W?Le?n?g?t?h?|"\W?Le?n?g?t?h?|\"xa0Le?n?g?t?h?|xa0Le?n?g?t?h?|"|Length','',str(x))).apply(lambda x: re.sub(r"\[|\]|'|\\|•",'',str(x))).apply(lambda x: re.sub(r"-",' ',str(x)))

# df_new['matches_length']=df_new['matches'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# # df_new['matches']=df_new['matches'].apply(lambda x: round_string_float(x))
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: re.sub('\\\" L',' in',str(x))).apply(lambda x: re.sub('\\\\','',str(x))).apply(lambda x: re.sub(',','","',str(x))).apply(lambda x: re.sub('" | "','"',str(x)))
# import ast
# def remove_duplicates(A):
#     [A.pop(count) for count,elem in enumerate(A) if A.count(elem)!=1]
#     return A
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(x)]))
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: str(x)).apply(lambda x: re.sub("'",'"',str(x))).apply(lambda x: re.sub('" | "','"',str(x))).apply(lambda x: re.sub('  ',' ',str(x))).apply(lambda x: re.sub('\”|\)|inig','',str(x))).apply(lambda x: re.sub('  ',' ',str(x)))
# # del df_new['matches_height']
# # del df_new['matches_ft']
# del df_new['new_matches']
# # del df_new['blank_matches']
# # del df_new['blank_matches_long']
# # del df_new['matches']

# df_new['matches_length']=df_new['matches_length'].apply(lambda x: remove_duplicates([n.strip() for n in ast.literal_eval(str(x))]))
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: natsorted(ast.literal_eval(re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))))
# df_new['matches_length']=df_new['matches_length'].apply(lambda x: re.sub(r"'",'',str(re.sub(r'""','"',str(re.sub(r'" ','"',str(re.sub(r'  ',' ',str(re.sub(r',','","',str(re.sub(r'\]','"]',str(re.sub(r'\]','"]',str(re.sub(r'\[','["',str(x)))))))))))))))))

In [44]:
# print(len(df_new))
# df_new[curation_col]=df_new['matches_length']
# matchesnew=df_new[['external_id',curation_col]]
# # df_new[0:500]

# NA Values-Ready

In [45]:
regex_pattern_blank=r'''(?i)(\d+\s?x\s?\d+)|(\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:"?i?n?c?h?'?'?f?o?o?t?f?e?e?t?”?).?x.?\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:"?i?n?c?h?'?'?f?o?o?t?f?e?e?t?”?)?.?x?.?\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:"?i?n?c?h?'?'?f?o?o?t?f?e?e?t?”?))|\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:"?i?n?c?h?'?'?f?o?o?t?f?e?e?t?”?)|()'''            
df['blank_matches'] = df['product_name'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
df['blank_matches_long'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))

In [46]:
regex_pattern_na=r'''(?i)((?<!Cord Measures.{10})\d+(?:(?:.\d+\/\d+)|(?:\.\d+))?.{0,2}["'].{0,2}[LWHD]?\b)|((?<!\.)\d+(?!\.\d).{0,3}?diam)|(measure)|()'''
regex_pattern_measure=r'(?i)(measure)|()'
df_na=df
df_na['na_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_na, x))
df_na['na_measure_matches'] = df_na['long_desc'].apply(lambda x: re_extract(regex_pattern_measure, x))
print(len(df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')]))
df_na=df_na[(df_na['na_matches'].astype(str)=='[]')&(df_na['na_measure_matches'].astype(str)=='[]')&(df_na['blank_matches'].astype(str)=='[]')]

# del df_na['custom_fields']
# del df['na_measure_matches']

df_na[curation_col]='n/a'
matchesna=df_na[['external_id',curation_col]]
# df_na[0:500]

112


# Seperating mm and inches for split values

In [47]:
# print(len(df))
# inch_ft=r'''\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:(?:inch)|(?:")|(?:(?<!')'(?!'))|(?:ft)|(?:”)|(?:''(?!\w)))?\W?[lwhd]?\b\W?x\W?\d+(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:(?:inch)|(?:")|(?:(?<!')'(?!'))|(?:ft)|(?:”)|(?:''(?!\w)))?\W?[lwhd]?\b\W?\W?x?\W?\d*(?:(?:.?\d+\/\d+)|(?:\.\d+))?\W?(?:(?:inch)|(?:")|(?:(?<!')'(?!'))|(?:ft)|(?:”)|(?:''(?!\w)))?\W?[lwhd]?\b|()'''
# df['inch_ft']=df['long_desc'].apply(lambda x: re_extract(inch_ft, str(x)))
# mm=r'''(?i)(\d+(?:\.\d+)?\W?mm)|()'''
# df['mm']=df['long_desc'].apply(lambda x: re_extract(mm, str(x)))


# print('With imperial: '+str(len(df[df['inch_ft'].astype(str)!='[]'])))
# print('With metric: '+str(len(df[df['mm'].astype(str)!='[]'])))
# print('Without metric and imperial: '+str(len(df[(df['mm'].astype(str)=='[]')&(df['inch_ft'].astype(str)=='[]')])))

# df_na_measure=df[(df['mm'].astype(str)=='[]')&(df['inch_ft'].astype(str)=='[]')]
# df_imp=df[df['mm'].astype(str)=='[]']
# df_mm=df[df['mm'].astype(str)!='[]']

# TWO Max Min (Length is the max)

In [48]:
# all_split=r'''(?i)(\d+\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b\W?x\W?\d+\W?\W?\d*\/?\W?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b\W?\W?x?\W?\d*\W?\d*\/?\d*\W?I?n?c?h?"?'?f?t?\W?[lwhd]?\b)|()'''
# pat_split_THREE=r'''(?i)(\[\"(\d+(\.\d*|\W\d+\/d*)?).?\w{0,4}.?x.?(\d+(\.\d*|\W\d+\/d*)?).?\w{0,4}.?x.?(\d+(\.\d*|\W?\d+\/d*)?).?\w{0,4}\"\])|()'''
# pat_split_TWO=r'''(?i)(\[\"\d+(?:\.?\d+)?\W{0,4}(?:(?:in)|(?:ft))?\"\s?x\s?\"\d+\.?\d*\W{0,4}(?:(?:in)|(?:ft))?\"\])|()'''

# df_imp['matches_split']=df_imp['product_name'].apply(lambda x: re_extract(all_split, str(x)))
# df_imp['matches_split']=df_imp['matches_split'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
# df_imp['matches_split']=df_imp['matches_split'].apply(lambda x: re.sub(r'''�''','',str(x))).apply(lambda x: re.sub(r'''(\[\"\[\')|(\[\'\[\")|(\[\"\[\\\')''',"['[\'",str(x))).apply(lambda x: re.sub(r'''(\'\]\"\])|(\\\'\]\"\])|(\"\]\'\])''','''\']']''',str(x))).apply(lambda x: re.sub(r'''\"''',' in',str(x))).apply(lambda x: re.sub(r'''(\[\'\[\\\')|(\[\'\[\')''','["',str(x))).apply(lambda x: re.sub(r'''(\\\'\]\'\])|(\'\]\'\])''','"]',str(x))).apply(lambda x: re.sub(r'''(?i)x''','"x"',str(x))).apply(lambda x: re.sub(r'''\'''',' ft',str(x))).apply(lambda x: re.sub(r''' ''','',str(x))).apply(lambda x: re.sub(r'''\\ft\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''\\ft''','ft',str(x))).apply(lambda x: re.sub(r'''X''','x',str(x))).apply(lambda x: re.sub(r'''in''',' in',str(x))).apply(lambda x: re.sub(r'''ft''',' ft',str(x))).apply(lambda x: re.sub(r'''\,.*''','"]',str(x))).apply(lambda x: re.sub(r'''\\\\.?ft''','ft',str(x)))                                                                                                                                                             

# df_imp['matches_split_three'] = df_imp['matches_split'].apply(lambda x: re_extract(pat_split_THREE, str(x)))
# df_imp['matches_split_two'] = df_imp['matches_split'].apply(lambda x: re_extract(pat_split_TWO, str(x)))

# regex_pattern_blank=r'(?i)(\d+\.\d*.?l\b)|()'
# df_imp['matches'] = df_imp['matches_split'].apply(lambda x: re_extract(regex_pattern_blank, str(x)))
# split=df_imp[df_imp['matches_split'].astype(str)!='[]']
# print(len(split))
# print(len(split[split['matches_split_two'].astype(str)!='[]']))
# print(len(split[split['matches_split_three'].astype(str)!='[]']))

In [49]:
# del split['matches_length']
# del split['matches_height']
# del split['matches_width']
# del split['matches_diameter']
# del split['new_matches']
# del split['matches_ft']
# del split['blank_matches']

In [50]:
# two=split[split['matches_split_two'].astype(str)!='[]']
# two['matches_split_two']=two['matches_split_two'].apply(lambda x: re.sub(r''' ''',r' ',str(x))).apply(lambda x: re.sub(r'''\[\'\[''','[',str(x))).apply(lambda x: re.sub(r'''\]\'\]''',']',str(x))).apply(lambda x: re.sub(r'''(?<!in)(?<!ft)"x"''',' in"x"',str(x))).apply(lambda x: re.sub(r'''(?<!in)(?<!ft)"]''',' in"]',str(x))).apply(lambda x: re.sub(r'''\[|\]|\?|\:|\)''','',str(x)))                               

# two['matches_split_two']=two['matches_split_two'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, str(x))])
# two['matches_split_two']=two['matches_split_two'].str.get(0).apply(lambda x: x.lower())
# two[['length', 'width']]=two['matches_split_two'].str.split("x",expand=True,)

### Split Inches

In [51]:
# split_inch=two
# pat_split=r'''(?i)(\binc?h?)|(")|(”)|((?<!\w)'{2})|(''(?!\w))|()'''
# pat_split_ft=r'''(?i)(feet)|(foot)|((?<!')(?!\w)'(?!\w)(?<!'))|(ft)|()'''

# split_inch['inch'] = split_inch['width'].apply(lambda x: re_extract(pat_split, str(x)))
# split_inch['inches'] = split_inch['length'].apply(lambda x: re_extract(pat_split, str(x)))
# split_inch['ft'] = split_inch['width'].apply(lambda x: re_extract(pat_split_ft, str(x)))
# split_inch['ftes'] = split_inch['length'].apply(lambda x: re_extract(pat_split_ft, str(x)))

# split_inch=split_inch[(split_inch['inch'].astype(str)!='[]')&(split_inch['inches'].astype(str)!='[]')&(split_inch['ft'].astype(str)=='[]')&(split_inch['ftes'].astype(str)=='[]')]

# split_inch['length']=split_inch['length'].apply(lambda x:re.sub(r'\s?in|\s?inch|ch','',str(x))).apply(lambda x: re.sub(r'"','',str(x)))
# split_inch['width']=split_inch['width'].apply(lambda x:re.sub(r'\s?in|\s?inch|ch','',str(x))).apply(lambda x: re.sub(r'"','',str(x)))


# split_inch['length']=split_inch['length'].astype(float)
# split_inch['width']=split_inch['width'].astype(float)
# split_inch['Max'] = split_inch[['length', 'width']].values.max(1)
# split_inch['Min'] = split_inch[['length', 'width']].values.min(1)
# split_inch['Max']=split_inch['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# print(len(split_inch))
# split_inch[curation_col]=split_inch['Max']
# split_inch[curation_col]=split_inch[curation_col].apply(lambda x: str(f"['{x} in']")).apply(lambda x: re.sub(r"'",'"',str(x)))
# matchsplitinch=split_inch[['external_id',curation_col]]
# # split_inch.head(2)

### Split Feet

In [52]:
# split_feet=two
# pat_split=r'''(?i)(\binc?h?)|(")|(”)|((?<!\w)'{2})|(''(?!\w))|()'''
# pat_split_ft=r'''(?i)(feet)|(foot)|((?<!')(?!\w)'(?!\w)(?<!'))|(ft)|()'''

# split_feet['inch'] = split_feet['width'].apply(lambda x: re_extract(pat_split, str(x)))
# split_feet['inches'] = split_feet['length'].apply(lambda x: re_extract(pat_split, str(x)))
# split_feet['ft'] = split_feet['width'].apply(lambda x: re_extract(pat_split_ft, str(x)))
# split_feet['ftes'] = split_feet['length'].apply(lambda x: re_extract(pat_split_ft, str(x)))

# split_feet=split_feet[(split_feet['ft'].astype(str)!='[]')&(split_feet['ftes'].astype(str)!='[]')&(split_feet['inches'].astype(str)=='[]')]

# split_feet['length']=split_feet['length'].apply(lambda x:re.sub(r'\s?ft','',str(x))).apply(lambda x: re.sub(r'"','',str(x)))
# split_feet['width']=split_feet['width'].apply(lambda x:re.sub(r'\s?ft','',str(x))).apply(lambda x: re.sub(r'"','',str(x)))


# split_feet['length']=split_feet['length'].astype(float)
# split_feet['width']=split_feet['width'].astype(float)
# split_feet['Max'] = split_feet[['length', 'width']].values.max(1)
# split_feet['Min'] = split_feet[['length', 'width']].values.min(1)
# split_feet['Max']=split_feet['Max'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))
# print(len(split_feet))
# split_feet[curation_col]=split_feet['Max']
# split_feet[curation_col]=split_feet[curation_col].apply(lambda x: str(f"['{x} ft']")).apply(lambda x: re.sub(r"'",'"',str(x)))
# matchsplitfeet=split_feet[['external_id',curation_col]]
# # split_feet.head(2)

In [53]:
# split_feet[0:500]

# Unitless

In [54]:
# unitless=df[(df['inches'].astype(str)=='[]')&(df['feet'].astype(str)=='[]')&(df['blank_matches'].astype(str)!='[]')]
# print('unmatched: '+str(len(df[(df['inches'].astype(str)=='[]')&(df['feet'].astype(str)=='[]')&(df['blank_matches'].astype(str)!='[]')])))
# print('Exploded Values: '+str(len(df[(df['inches'].astype(str)=='[]')&(df['feet'].astype(str)=='[]')&(df['blank_matches'].astype(str)!='[]')]['blank_matches'].explode().value_counts())))
# unitless['blanks']=unitless['blank_matches'].apply(lambda x: re.sub(r'\�|\,|•','',str(x)))
# unitless['blanks']=unitless['blanks'].apply(lambda x:[re.sub(r'(?:(\d+)[-\s])?(\d+/\d+)', frac2string, x)])
# unitless['blanks']=unitless['blanks'].apply(lambda x: re.sub(r'''\[\"\[\'|\'\]\"\]''','',str(x)))
# # unitless[['length', 'width']]=unitless['blanks'].str.split("x",expand=True,)
# unitless

In [55]:
# df[(df['inches'].astype(str)=='[]')&(df['feet'].astype(str)=='[]')]['blank_matches_long'].explode().value_counts()[0:500]

In [56]:
# # def get_df_name(df):
# #     name =[x for x in globals() if globals()[x] is df][0]
# #     return name
# # print(get_df_name(matchsplitinch))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchcustom))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][0]
#     return name
# print(get_df_name(matchesna))


# # def get_df_name(df):
# #     name =[x for x in globals() if globals()[x] is df][0]
# #     return name
# # print(get_df_name(matchesheight))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matchnote))

def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name
print(get_df_name(matchcust))

# def get_df_name(df):
#     name =[x for x in globals() if globals()[x] is df][1]
#     return name
# print(get_df_name(matchone))

matchcust


In [57]:
stop 

NameError: name 'stop' is not defined

# send to the folder for upload

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchesna)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchlength)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
looks_good('WBMason', attribute, df, matchcust)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchpaperfour)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][1]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchone)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchnote)

In [ ]:
def get_df_name(df):
    name =[x for x in globals() if globals()[x] is df][0]
    return name

def looks_good(customer, attribute, df, matches): 
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload' 
#     non_matches = df[df['matches'].astype(str) == '[]']
#     non_matches[curation_col] = r'n/a'
#     non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/WBMason - {attribute}-{get_df_name(matches)}-matches.csv',index=False) 
    
looks_good('WBMason', attribute, df, matchdiam)